[![Lab Documentation and Solutions](https://img.shields.io/badge/Lab%20Documentation%20and%20Solutions-purple)](https://mongodb-developer.github.io/ai-rag-lab/)


# Step 1: Setup prerequisites

In [1]:
!pip install -Uq pandas datasets


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [2]:
import os
from pymongo import MongoClient
from utils import track_progress

import pandas as pd
pd.set_option('display.max_colwidth', 150)

from datasets import load_dataset
from typing import Dict, List

/Users/wenjie.teo/Documents/repositories/genai-devday-notebooks/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# If you are using your own MongoDB Atlas cluster, use the connection string for your cluster here
MONGODB_URI = "mongodb://localhost:50566/?directConnection=true"
# Initialize a MongoDB Python client
mongodb_client = MongoClient(MONGODB_URI)
# Check the connection to the server
mongodb_client.admin.command("ping")

{'ok': 1.0,
 '$clusterTime': {'clusterTime': Timestamp(1759076643, 1),
  'signature': {'hash': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00',
   'keyId': 0}},
 'operationTime': Timestamp(1759076643, 1)}

In [ ]:
PROXY_ENDPOINT = os.environ.get("PROXY_ENDPOINT")
PASSKEY=

In [7]:
# Define database and collection names
DB_NAME = "virtual_primary_care_assistant"
DRUG_REVIEW_COLLECTION_NAME = "drug_reviews"

# Get a reference to the database (creates it if it doesn't exist)
db = mongodb_client[DB_NAME]

# Get a reference to collections for later use
drug_reviews_collection = db[DRUG_REVIEW_COLLECTION_NAME]

# Step 2: Load drug reviews dataset

In [8]:
# Load the drug reviews dataset from Hugging Face repository
# This dataset contains patient reviews of various medications
# 'Reboot87/drugs_reviews_dataset' contains structured data about drug experiences
drug_reviews_dataset = load_dataset(
    "Reboot87/drugs_reviews_dataset", streaming=True, split="train"
)

# Limit the dataset to 10,000 examples to manage memory usage and processing time
# This sample size should be sufficient for building our demonstration model
# The streaming=True parameter ensures we don't load the entire dataset into memory
drug_reviews_dataset = drug_reviews_dataset.take(1000)

# Similarly convert the drug reviews dataset to a DataFrame
# This enables SQL-like operations, filtering, and statistical analysis
# Having both datasets as DataFrames ensures consistent data handling approaches
drug_reviews_dataset = pd.DataFrame(drug_reviews_dataset)

# Remove the attributes patientId, date, usefulCount and review_length from the drug_reviews_dataset
# patientId: Removed to ensure data anonymization and privacy protection
# date: Temporal information isn't critical for our current analysis
# usefulCount: Engagement metrics aren't relevant for our semantic understanding
# review_length: This is a derived feature that can be recalculated if needed
drug_reviews_dataset = drug_reviews_dataset.drop(
    columns=["patientId", "date", "usefulCount", "review_length"]
)

In [9]:
# Preview a document to understand its structure
drug_reviews_dataset.head()

,drugName,condition,review,rating
0,cyclosporine,keratoconjunctivitis sicca,"""I have used Restasis for about a year now and have seen almost no progress. For most of my life I've had red and bothersome eyes. After trying v...",2
1,etonogestrel,birth control,"""My experience has been somewhat mixed. I have been using Implanon now for nearly 14 months and have decided to get it removed because I bleed eve...",7
2,implanon,birth control,"""This is my second Implanon would not recommend at all....first one was okay for the first 2 years until I started bleeding which never stopped wh...",1
3,hydroxyzine,anxiety,"""I recommend taking as prescribed, and the bottle usually says ""take X amount every X hours"". I think that having a steady stream of any medicatio...",10
4,dalfampridine,multiple sclerosis,"""I have been on Ampyra for 5 days and have been so happy with this new pill. The first 2 days were NOT good with the side effects, but each day ju...",9


In [10]:
# Delete existing records
drug_reviews_collection.delete_many({});

# Insert new records
drug_reviews_dataset = drug_reviews_dataset.to_dict("records")

#<CODE_BLOCK_1>
drug_reviews_collection.insert_many(drug_reviews_dataset)

InsertManyResult([ObjectId('68d9618ef39f27886354e76e'), ObjectId('68d9618ef39f27886354e76f'), ObjectId('68d9618ef39f27886354e770'), ObjectId('68d9618ef39f27886354e771'), ObjectId('68d9618ef39f27886354e772'), ObjectId('68d9618ef39f27886354e773'), ObjectId('68d9618ef39f27886354e774'), ObjectId('68d9618ef39f27886354e775'), ObjectId('68d9618ef39f27886354e776'), ObjectId('68d9618ef39f27886354e777'), ObjectId('68d9618ef39f27886354e778'), ObjectId('68d9618ef39f27886354e779'), ObjectId('68d9618ef39f27886354e77a'), ObjectId('68d9618ef39f27886354e77b'), ObjectId('68d9618ef39f27886354e77c'), ObjectId('68d9618ef39f27886354e77d'), ObjectId('68d9618ef39f27886354e77e'), ObjectId('68d9618ef39f27886354e77f'), ObjectId('68d9618ef39f27886354e780'), ObjectId('68d9618ef39f27886354e781'), ObjectId('68d9618ef39f27886354e782'), ObjectId('68d9618ef39f27886354e783'), ObjectId('68d9618ef39f27886354e784'), ObjectId('68d9618ef39f27886354e785'), ObjectId('68d9618ef39f27886354e786'), ObjectId('68d9618ef39f27886354e7

# Step 3: Implement search on drug reviews

### Create a search index

📚 https://www.mongodb.com/docs/atlas/atlas-search/define-field-mappings/


In [11]:
# Define the text search index definition for the drugs_review dataset.
# This configuration specifies that only the "drugName" and "condition" fields will be indexed,
# and automatic field detection is disabled.
DRUG_REVIEW_SEARCH_INDEX_NAME='drug_review_search_index'

drug_review_text_search_model = {
    "name": DRUG_REVIEW_SEARCH_INDEX_NAME,
    "type": "search",
    "definition": {
        "analyzer": "lucene.english",
        "mappings": {
            "dynamic": False,  # Disable automatic detection; only explicitly defined fields are indexed.
            #<CODE_BLOCK_2>
            "fields": {
                "drugName": {
                    "type": "string"
                },  # Index the "drugName" field as searchable text.
                "condition": {
                    "type": "string"
                },  # Index the "condition" field as searchable text.
            }
        }
    }
}

In [12]:
# create search index
# <CODE_BLOCK_3>
drug_reviews_collection.create_search_index(drug_review_text_search_model)

'drug_review_search_index'

### Perform text search

📚 https://www.mongodb.com/docs/atlas/atlas-search/aggregation-stages/search/


In [13]:
# Definition to perform text search
def drug_review_text_search(query_text):
    """
    Perform a text search in the MongoDB collection based on the user query.

    Args:
        query_text (str): The user's query string.

    Returns:
        list: A list of matching documents.
    """

    # Define the text search stage using MongoDB's $search operator.
    # This is part of Atlas Search and provides more powerful text search capabilities
    # than MongoDB's standard text index.
    # Perform a text search on the paths "drugName" and condition.
    # Cater for typo tolerance with 2 maxEdits.
    text_search_stage = {
        "$search": {
            "index": DRUG_REVIEW_SEARCH_INDEX_NAME,
            #<CODE_BLOCK_4>
            "text": {
                "query": query_text,  # The actual search term provided by the user.
                "path": ["drugName","condition"], 
                "fuzzy": {
                    "maxEdits": 2
                }
            },
        }
    }

    # Limit the number of results returned to improve performance.
    # This is especially important for large collections.
    limit_stage = {"$limit": 5}

    # Define which fields to include in the returned documents.
    # Excluding unnecessary fields reduces bandwidth and processing overhead.
    project_stage = {
        "$project": {
            "_id": 0,  # Exclude MongoDB's internal ID field.
            "embedding": 0,  # Exclude the embedding field.
        }
    }

    # Combine all stages into a MongoDB aggregation pipeline.
    # The pipeline will execute stages in sequence: search, limit, then project.
    pipeline = [text_search_stage, limit_stage, project_stage]

    # Execute the search by running the aggregation pipeline against the specified collection.
    # Convert the cursor to a list to ensure results are fully fetched before the function returns.
    #results = <CODE_BLOCK_5>
    results = drug_reviews_collection.aggregate(pipeline)

    return list(results)

In [14]:
temp_results = drug_review_text_search("I am coughing")
pd.DataFrame(temp_results).head()

,drugName,condition,review,rating
0,dextromethorphan,cough,"""I got some relief from constant dry cough but I wouldn't say it's brilliant although I am asthmatic too. This cough however was not my normal n...",5
1,tessalon perles,cough,"""Have been prescribed tessalon perles several times for bronchitis and they've never once worked for me. I might as well eat a sugar cube. Sadly c...",1
2,delsym,cough,"""I took this late at night expecting it to work and I should have read the reviews first it didn't stop my cough at all and I think made it worse ...",1
3,benzonatate,cough,"""My doctor said the pearls did not work for her but other people said they work great. I tried them and in 20 min my cough was gone!! I see they d...",10
4,dextromethorphan,cough,"""Did absolutely nothing to alleviate my cough. And now I am feeling slightly nauseated. I actually think my cough was better before taking it, as ...",1


In [15]:
# test our text search by mispelling dextromethorphan
temp_results = drug_review_text_search("How is dextormethorphan")

# notice the false positives
pd.DataFrame(temp_results).head()

,drugName,condition,review,rating
0,dextromethorphan,cough,"""I got some relief from constant dry cough but I wouldn't say it's brilliant although I am asthmatic too. This cough however was not my normal n...",5
1,dextromethorphan,cough,"""Did absolutely nothing to alleviate my cough. And now I am feeling slightly nauseated. I actually think my cough was better before taking it, as ...",1
2,atripla,hiv infection,"""Every night at bedtime I take my medication. sometimes its hard to get up and walk to the counter and get a glass of water to take a pill that is...",10
3,atripla,hiv infection,"""Diagnosed in July 2016. I suspected it was HIV when I broke out in several(roughly 12) canker sores in Mouth. I googled this and HIV was listed a...",10
4,stribild,hiv infection,"""I have been on Stribild since April 2013 and started with a 57,500 viral load and 328 cd4. Within a month my viral load was down to 25! As of O...",10


# Step 4: Generate embeddings on drug reviews

In [16]:
# You may see a warning upon running this cell. You can ignore it.
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

In [17]:
# Load the `gte-small` model using the Sentence Transformers library
embedding_model = SentenceTransformer("thenlper/gte-small")

📚 https://huggingface.co/thenlper/gte-small#usage (See "Use with sentence-transformers" under Usage)

In [18]:
# Define a function that takes a piece of text (`text`) as input, embeds it using the `embedding_model` instantiated above and returns the embedding as a list
# An array can be converted to a list using the `tolist()` method
def get_embedding(text: str) -> List[float]:
    """
    Generate the embedding for a piece of text.

    Args:
        text (str): Text to embed.

    Returns:
        List[float]: Embedding of the text as a list.
    """
    #embedding = <CODE_BLOCK_6>
    embedding = embedding_model.encode(text)
    return embedding.tolist()

In [19]:
embedded_docs = []
# Add an `embedding` field to each dictionary in `drug_reviews_dataset`
# The `embedding` field should correspond to the embedding of the value of the `review` field
# Use the `get_embedding` function defined above to generate the embedding
# Append the updated dictionaries to `embedded_docs` initialized above.
for doc in tqdm(drug_reviews_dataset):

    # format doc to proper sentence
    record = (
        f"Drug Name: {doc['drugName']}. "
        f"Condition: {doc['condition']}. "
        f"Review: {doc['review']}."
    )
    # <CODE_BLOCK_7>
    doc['embedding'] = get_embedding(record)
    embedded_docs.append(doc)

  0%|          | 0/1000 [00:00<?, ?it/s]/Users/wenjie.teo/Documents/repositories/genai-devday-notebooks/.venv/lib/python3.13/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
100%|██████████| 1000/1000 [00:31<00:00, 31.34it/s]


In [20]:
# Check that the length of `embedded_docs` should be same as the dataset
len(embedded_docs)

1000

# Step 5: Re-ingest data into MongoDB


In [21]:
drug_reviews_collection.delete_many({})

DeleteResult({'n': 1000, 'electionId': ObjectId('7fffffff0000000000000003'), 'opTime': {'ts': Timestamp(1759076911, 100), 't': 3}, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1759076911, 100), 'signature': {'hash': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00', 'keyId': 0}}, 'operationTime': Timestamp(1759076911, 100)}, acknowledged=True)

In [22]:
# Bulk insert `embedded_docs` into the `collection` defined above -- should be a one-liner
drug_reviews_collection.insert_many(embedded_docs)
print(f"Ingested {drug_reviews_collection.count_documents({})} documents into the {DRUG_REVIEW_COLLECTION_NAME} collection.")

Ingested 1000 documents into the drug_reviews collection.


# Step 6: Implement vector search capability

### Create a vector search index

📚 https://www.mongodb.com/docs/atlas/atlas-vector-search/vector-search-type/


In [23]:
# Create vector index definition specifying:
# path: Path to the embeddings field
# numDimensions: Number of embedding dimensions- depends on the embedding model used
# similarity: Similarity metric. One of cosine, euclidean, dotProduct.
DRUG_REVIEW_VECTOR_SEARCH_INDEX_NAME='drug_review_vector_search_index'

drug_review_vector_search_model = {
    "name": DRUG_REVIEW_VECTOR_SEARCH_INDEX_NAME,
    "type": "vectorSearch",
    "definition": {
        "fields": [
            # Define a vector field on the path "embedding"
            # It has 384 dimensions and uses the cosine similarity
            # <CODE_BLOCK_8>
            {
                "type": "vector",
                "path": "embedding",
                "numDimensions": 384,
                "similarity": "cosine",
            }
        ]
    },
}

In [24]:
# create vector search index
# <CODE_BLOCK_9>
drug_reviews_collection.create_search_index(drug_review_vector_search_model)

'drug_review_vector_search_index'

In [25]:
# Use the `check_index_ready` function from the `utils` module to verify that the index was created and is in READY status before proceeding
from utils import check_index_ready
check_index_ready(drug_reviews_collection, DRUG_REVIEW_VECTOR_SEARCH_INDEX_NAME)

drug_review_vector_search_index index status: READY
drug_review_vector_search_index index definition: {'fields': [{'type': 'vector', 'path': 'embedding', 'numDimensions': 384, 'similarity': 'cosine'}]}


### Define a vector search function

📚 https://www.mongodb.com/docs/atlas/atlas-vector-search/vector-search-stage/#ann-examples (Refer to the "Basic Example")


In [27]:
# Define a function to retrieve relevant documents for a user query using vector search
def drug_review_vector_search(user_query: str) -> List[Dict]:
    """
    Retrieve relevant documents for a user query using vector search.

    Args:
    user_query (str): The user's query string.

    Returns:
    list: A list of matching documents.
    """

    # Generate embedding for the `user_query` using the `get_embedding` function defined in Step 4
    query_embedding = get_embedding(user_query)

    # Define an aggregation pipeline consisting of a $vectorSearch stage, followed by a $project stage
    # Set the number of candidates to 50 and only return the top 5 documents from the vector search
    # NOTE: Use variables defined previously for the `index`, `queryVector` and `path` fields in the $vectorSearch stage
    pipeline = [
    {
        #vector search stage
        #<CODE_BLOCK_10>
        "$vectorSearch": {
            "index": DRUG_REVIEW_VECTOR_SEARCH_INDEX_NAME,
            "queryVector": query_embedding,
            "path": "embedding",
            "numCandidates": 50,
            "limit": 5
        }
    },
    {
        "$project": {
            "_id": 0,
            "embedding": 0,
            "score": {"$meta": "vectorSearchScore"}
        }
    }
]

    # Execute the aggregation `pipeline` and store the results in `results`
    results = drug_reviews_collection.aggregate(pipeline)
    return list(results)

### Run vector search queries


In [28]:
temp_results = drug_review_vector_search("How is dextromethorphan")

pd.DataFrame(temp_results).head()

,drugName,condition,review,rating,score
0,dextromethorphan,cough,"""I got some relief from constant dry cough but I wouldn't say it's brilliant although I am asthmatic too. This cough however was not my normal n...",5,0.948889
1,dextromethorphan,cough,"""Did absolutely nothing to alleviate my cough. And now I am feeling slightly nauseated. I actually think my cough was better before taking it, as ...",1,0.939754
2,dextroamphetamine,narcolepsy,"""I am at the maximum amount per day 60mg. It has helped to a certain extent. I do not need a nap to make it through a normal work day. I still hav...",5,0.925711
3,dexilant,erosive esophagitis,"""I've been taking Dexilant 60 mg. for 10 days now. It was given to me because I felt like something was stuck in my throat.....esophagitis. Well...",10,0.924939
4,xiidra,dry eye disease,"""I have been using Xiidra for about a month for chronic dry eye disease and chronic keratocojunctivitis. I also use a steroid twice a day. The bur...",9,0.922210


# Step 7: Implementing hybrid search

In [41]:
def hybrid_search(
    text_search_query,
    vector_search_query,
    vector_weight=0.5,
    full_text_weight=0.5,
    top_k=10,
    text_search_paths=["drugName","condition"],
):
    """
    Conduct a hybrid search on a MongoDB Atlas collection that combines a vector search
    and a full-text search using Atlas Search.

    Args:
        user_query (str): The user's query string.
        collection (MongoCollection): The MongoDB collection to search.
        vector_search_index_name (str): The name of the vector search index.
        text_search_index_name (str): The name of the text search index.
        vector_weight (float): The weight of the vector search.
        full_text_weight (float): The weight of the full-text search.
        top_k (int): Number of results to return.

    Returns:
        list: A list of documents (dict) with combined scores.
    """

    # Get the pre-computed embedding vector for the user's query
    query_vector = get_embedding(vector_search_query)

    # Create a MongoDB aggregation pipeline to perform hybrid search
    vsPipeline = [{
                    "$vectorSearch": {
                        "index": DRUG_REVIEW_VECTOR_SEARCH_INDEX_NAME,  # Name of the vector search index
                        "path": "embedding",  # Field containing document embeddings
                        "queryVector": query_vector,  # The query vector to compare against
                        "numCandidates": top_k*10,  # Number of candidates to consider for similarity
                        "limit": top_k,  # Initial limit of results
                    }
                }]
    
    ftsPipeline = [{
                    "$search": {
                        "index": DRUG_REVIEW_SEARCH_INDEX_NAME,  # Name of the text search index
                        "text": {
                            "query": text_search_query,  # Raw text query from user
                            "path": text_search_paths,  # Field to search in
                            "fuzzy": {
                                "maxEdits": 2
                            }
                        }
                    }
                },{
                    "$limit": top_k
                }]

    rankFusion = {
        "$rankFusion": {
            "input": {
                "pipelines": {
                    "vs": vsPipeline,
                    "fts": ftsPipeline
                }
            },
            "combination": {
                "weights": {
                    "vs": 0.5,
                    "fts": 0.5
                },     
            },
            "scoreDetails": True
        }
    }

    pipeline = [
        rankFusion,
        { "$limit": top_k },
        {
            "$project": {
                "_id": 0,
                "scoreDetails": {"$meta": "scoreDetails"}
            }
        }
    ]

    # Execute the aggregation pipeline and convert results to a list
    results = list(drug_reviews_collection.aggregate(pipeline))
    return results

In [42]:
# query_text = "I have a cough, what drug would be best?"
query_text = "How is dextromethorphan?"

# Execute a hybrid search that combines both vector (semantic) and full-text search
# We can change the weightage if necessary

drug_reviews_hybrid_results = hybrid_search(
    query_text,  # Our natural language query
    query_text
)

pd.DataFrame(drug_reviews_hybrid_results).head()

/Users/wenjie.teo/Documents/repositories/genai-devday-notebooks/.venv/lib/python3.13/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


,scoreDetails
0,"{'value': 0.01639344262295082, 'description': 'value output by reciprocal rank fusion algorithm, computed as sum of (weight * (1 / (60 + rank))) a..."
1,"{'value': 0.01626123744050767, 'description': 'value output by reciprocal rank fusion algorithm, computed as sum of (weight * (1 / (60 + rank))) a..."
2,"{'value': 0.007936507936507936, 'description': 'value output by reciprocal rank fusion algorithm, computed as sum of (weight * (1 / (60 + rank))) ..."
3,"{'value': 0.007936507936507936, 'description': 'value output by reciprocal rank fusion algorithm, computed as sum of (weight * (1 / (60 + rank))) ..."
4,"{'value': 0.007936507936507936, 'description': 'value output by reciprocal rank fusion algorithm, computed as sum of (weight * (1 / (60 + rank))) ..."


# Step 8: Build the RAG application


In [43]:
import requests

### Define a function to create the chat prompt

In [44]:
def format_context(context) -> str:
    formatted_context = ""

    # Check if any documents were retrieved.
    if context and len(context) > 0:
        # Add a header for the context section.
        formatted_context = "\n\nRelevant information from drug reviews:\n\n"

        # Process each retrieved document and format its content.
        for i, doc in enumerate(context):
            # Extract key fields from the document.
            review = doc.get("review", "No review available")
            condition = doc.get("condition", "No condition available")
            drug_name = doc.get("drugName", "No drug name available")

            # Append the formatted document with a citation reference.
            formatted_context += f"[{i+1}] Review: {review}\nCondition: {condition}\nDrug Name: {drug_name}\n\n"
    return formatted_context

In [45]:
# Define a function to create the user prompt for our RAG application
def create_prompt(user_query: str) -> str:
    """
    Create a chat prompt that includes the user query and retrieved context.

    Args:
        user_query (str): The user's query string.

    Returns:
        str: The chat prompt string.
    """
    # Retrieve the most relevant documents for the `user_query` using the `vector_search` function defined in Step 7
    context = hybrid_search(user_query,user_query)
    # 1. Join the retrieved documents into a single string, where each document is separated by two new lines ("\n\n")
    # 2. Format the retrieved documents into context for the LLM.
    #formatted_context = <CODE_BLOCK_11>
    formatted_context = format_context(context)

    # 3. Craft the prompt for the LLM using the user query and the formatted context.
    prompt = f"""
Based on the following information, please answer the user's question:
User Question: {user_query}
{formatted_context}
Please provide a comprehensive answer based on the information above.
If the provided information does not contain the answer, state that clearly.
Include citation numbers [X] to indicate which sources were used for specific details.
"""

    # Prompt consisting of the question and relevant context to answer it
    return prompt

### Define a function to answer user queries

In [46]:
# Define a function to answer user queries
def generate_answer(user_query: str) -> None:
    """
    Generate an answer to the user query.

    Args:
        user_query (str): The user's query string.
    """
    # Use the `create_prompt` function above to create a chat prompt
    prompt = create_prompt(user_query)
    # Format the message to the LLM in the format [{"role": <role_value>, "content": <content_value>}
    # The role value for user messages must be "user"
    # Use the `prompt` created above to populate the `content` field in the chat message
    #messages = <CODE_BLOCK_12>
    messages = [{"role":"user","content":prompt}]
    # Send the chat messages to a serverless function to get back an LLM response
    response = requests.post(url=SERVERLESS_URL, json={"task": "completion", "data": messages})
    # Print the final answer
    print(response.json()["text"])

### Query the RAG application


In [47]:
query_text = "I have a cough, what drug would be best?"
#query_text = "How is dextromethorphan?"

generate_answer(query_text)

/Users/wenjie.teo/Documents/repositories/genai-devday-notebooks/.venv/lib/python3.13/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


MissingSchema: Invalid URL 'None': No scheme supplied. Perhaps you meant https://None?

# 🦹‍♀️ "Agentic" Pipeline


In [45]:
# Use LLM to filter out terms that are medical conditions, symptoms, or drug names
def get_terms_from_query(user_query: str) -> str:
    prompt = f"""
        For the following query, return me only words associated with medical conditions, symptoms or drug names: {user_query}.
        Respond only with the words, space separated.
        """

    messages = [{"role":"user","content":prompt}]
    # Send the chat messages to a serverless function to get back an LLM response
    
    response = requests.post(url=SERVERLESS_URL, json={"task": "completion", "data": messages})
    return response.json()["text"]

In [ ]:
# test the definition
get_terms_from_query("I have cough and sore throat")

In [47]:
def get_medication_reviews(user_query:str) -> str:
    """
    Retrieves patient reviews and information about medications related to the query.

    This tool searches a database of medication reviews to find relevant patient experiences
    with drugs that match the symptoms, conditions, or medication names in the user query.
    Use this tool when discussing specific medications or treatment options.

    Args:
        user_query (str): The medication name, condition, or symptom to search for reviews about.

    Returns:
        str: Patient reviews and experiences with relevant medications.
    """
    
    refined_text_search_query = get_terms_from_query(user_query)
    
   # Execute the hybrid search to find medication reviews
    retrieved_context = hybrid_search(
        refined_text_search_query,
        user_query,
    )

    return retrieved_context

In [ ]:
temp_results = get_medication_reviews("I have a cought and sore throat, what is best for me")

pd.DataFrame(temp_results).head()

In [50]:
# Define a function to create the user prompt for our RAG application
def create_prompt(user_query: str) -> str:
    """
    Create a chat prompt that includes the user query and retrieved context.

    Args:
        user_query (str): The user's query string.

    Returns:
        str: The chat prompt string.
    """
    # Retrieve the most relevant documents for the `user_query` using the `vector_search` function defined in Step 7
    context = get_medication_reviews(user_query)
    # Join the retrieved documents into a single string, where each document is separated by two new lines ("\n\n")
        # 2. Format the retrieved documents into context for the LLM.
    formatted_context = format_context(context)

    # 3. Craft the prompt for the LLM using the user query and the formatted context.
    prompt = f"""
Based on the following information, please answer the user's question:
User Question: {user_query}
{formatted_context}
Please provide a comprehensive answer based on the information above.
If the provided information does not contain the answer, state that clearly.
Include citation numbers [X] to indicate which sources were used for specific details.
"""

    # Prompt consisting of the question and relevant context to answer it
    return prompt

In [ ]:
# Use this if you have not completed "adding memory"
# generate_answer("I have a flu")

# Use this if you have completed "adding memory"
generate_answer("session", "I have a flu")

# Step 9: Add memory to the RAG application


In [37]:
from datetime import datetime

In [38]:
history_collection = mongodb_client[DB_NAME]["chat_history"]

📚 https://pymongo.readthedocs.io/en/stable/api/pymongo/collection.html#pymongo.collection.Collection.create_index


In [ ]:
# Create an index on the key `session_id` for the `history_collection` collection
history_collection.create_index("session_id")

### Define a function to store chat messages in MongoDB

📚 https://pymongo.readthedocs.io/en/stable/api/pymongo/collection.html#pymongo.collection.Collection.insert_one

In [40]:
def store_chat_message(session_id: str, role: str, content: str) -> None:
    """
    Store a chat message in a MongoDB collection.

    Args:
        session_id (str): Session ID of the message.
        role (str): Role for the message. One of `system`, `user` or `assistant`.
        content (str): Content of the message.
    """
    # Create a message object with `session_id`, `role`, `content` and `timestamp` fields
    # `timestamp` should be set the current timestamp
    message = {
        "session_id": session_id,
        "role": role,
        "content": content,
        "timestamp": datetime.now(),
    }
    # Insert the `message` into the `history_collection` collection
    history_collection.insert_one(message)

### Define a function to retrieve chat history from MongoDB

📚 https://pymongo.readthedocs.io/en/stable/api/pymongo/cursor.html#pymongo.cursor.Cursor.sort

In [41]:
def retrieve_session_history(session_id: str) -> List:
    """
    Retrieve chat message history for a particular session.

    Args:
        session_id (str): Session ID to retrieve chat message history for.

    Returns:
        List: List of chat messages.
    """
    # Query the `history_collection` collection for documents where the "session_id" field has the value of the input `session_id`
    # Sort the results in increasing order of the values in `timestamp` field
    cursor = history_collection.find({"session_id": session_id}).sort("timestamp", 1)

    if cursor:
        # Iterate through the cursor and extract the `role` and `content` field from each entry
        # Then format each entry as: {"role": <role_value>, "content": <content_value>}
        messages = [{"role": msg["role"], "content": msg["content"]} for msg in cursor]
    else:
        # If cursor is empty, return an empty list
        messages = []

    return messages

### Handle chat history in the `generate_answer` function

In [42]:
def generate_answer(session_id: str, user_query: str) -> None:
    """
    Generate an answer to the user's query taking chat history into account.

    Args:
        session_id (str): Session ID to retrieve chat history for.
        user_query (str): The user's query string.
    """
    # Initialize list of messages to pass to the chat completion model
    messages = []

    # Retrieve documents relevant to the user query and convert them to a single string
    prompt = create_prompt(user_query)
    # Create a system prompt containing the retrieved context
    system_message = {
        "role": "user",
        "content": prompt
    }
    # Append the system prompt to the `messages` list
    messages.append(system_message)

    # Use the `retrieve_session_history` function to retrieve message history from MongoDB for the session ID `session_id` 
    # And add all messages in the message history to the `messages` list 
    message_history = retrieve_session_history(session_id)
    messages.extend(message_history)

    # Format the user query in the format {"role": <role_value>, "content": <content_value>}
    # The role value for user messages must be "user"
    # And append the user message to the `messages` list
    user_message = {"role": "user", "content": user_query}
    messages.append(user_message)

    # Send the chat messages to a serverless function to get back an LLM response
    response = requests.post(url=SERVERLESS_URL, json={"task": "completion", "data": messages})

    # Extract the answer from the response
    answer = response.json()["text"]

    # Use the `store_chat_message` function to store the user message and also the generated answer in the message history collection
    # The role value for user messages is "user", and "assistant" for the generated answer
    store_chat_message(session_id, "user", user_query)
    store_chat_message(session_id, "assistant", answer)

    print(answer)

In [ ]:
generate_answer(
    session_id="1",
    user_query="I have a sore throat",
)

In [ ]:
generate_answer(
    session_id="1",
    user_query="What if i have a cough too",
)